In [4]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import plotly.express as px

mpl.rc('font', family = 'Malgun Gothic')

In [5]:
df1 = pd.read_csv('01_Data.csv')
df1.head()

,Index,Member_ID,Sales_Type,Contract_Type,Channel,Datetime,Term,Payment_Type,Product_Type,Amount_Month,Customer_Type,Age,Address1,Address2,State,Overdue_count,Overdue_Type,Gender,Credit_Rank,Bank
0,1,66758234,렌탈,일반계약,영업방판,2019-05-06,60,CMS,DES-1,96900,개인,42.0,경기도,경기도,계약확정,0,없음,여자,9.0,새마을금고
1,2,66755948,렌탈,교체계약,영업방판,2020-02-20,60,카드이체,DES-1,102900,개인,39.0,경기도,경기도,계약확정,0,없음,남자,2.0,현대카드
2,3,66756657,렌탈,일반계약,홈쇼핑/방송,2019-02-28,60,CMS,DES-1,96900,개인,48.0,경기도,경기도,계약확정,0,없음,여자,8.0,우리은행
3,4,66423450,멤버십,멤버십3유형,재계약,2019-05-13,12,CMS,DES-1,66900,개인,39.0,경기도,경기도,계약확정,0,없음,남자,5.0,농협회원조합
4,5,66423204,멤버십,멤버십3유형,재계약,2019-05-10,12,CMS,DES-1,66900,개인,60.0,경기도,경기도,기간만료,12,있음,남자,8.0,농협회원조합


# 지도 학습 절차
- 계약 해지 여부 'State' 항목을 목표변수(Y)로, 계약정보(X)를 이용해 예측(분류)하는 모델을 생성

1. 데이터 핸들링 : 데이터를 불러와서 파생변수 생성 / 이상치 제거 / 변수 선택하는 작업
2. 목표변수(Y)와 설명변수(X)를 선언
3. 학습 데이터(Train Set)와 검증 데이터(Test Set)로 분할
4. 학습을 수행 (Train Set)
   - 특성 공학 (Feature Engineering / 학습을 잘 수행하기 위한 데이터 처리기법)
     (결측값 처리 / 스케일링 / 인코딩 / 교차검증 / ...)
   - 학습 알고리즘에 의한 학습을 수행
5. 평가 (Evaluation)
   - 학습 능력 평가
   - 일반화 능력 평가 (Test Set)
6. 실행
   - Model을 사용 가능한 형태로 변환 (파일 / 새로운 데이터 입력)

In [6]:
# 1. 데이터 핸들링
# 'State' 항목을 '해약' : 1 / '정상' : 0 데이터로 변환
df1['State'].value_counts()

State
계약확정     50620
해약확정       622
기간만료        45
해약진행중       14
Name: count, dtype: int64

In [7]:
# replace() : 특정 값을 다른 값으로 변환
df1['Target'] = df1['State'].replace({'계약확정' : 0, '해약확정' : 1,
                                      '기간만료' : 0, '해약진행중' : 1})
df1['Target'].value_counts()

C:\Users\magne\AppData\Local\Temp\ipykernel_26652\4082301724.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1['Target'] = df1['State'].replace({'계약확정' : 0, '해약확정' : 1,


Target
0    50665
1      636
Name: count, dtype: int64

In [8]:
# 결측값 제거
df2 = df1.dropna()

In [9]:
# 2. 목표변수(Y)와 설명변수(X)를 선언
Y = df2['Target'] # 목표변수 (Y)
X = df2[['Term', 'Amount_Month', 'Age', 'Credit_Rank']] # 설명변수 (X)

In [10]:
# 3. 학습 데이터와 검증 데이터를 분할
from sklearn.model_selection import train_test_split
# from : 라이브러리 내 특정 클래스(특정 동작을 수행하는 함수들이 모여 있는)를 추출

In [33]:
# train_test_split() : 특정 데이터를 무작위로 나누는 함수 (8:2 비율로 나눔)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1234)
# random_state = 입력한 수가 동일한 값에 대해 랜덤하게 분할

In [23]:
# 4. 학습을 수행
from sklearn.tree import DecisionTreeClassifier # 분류학습을 도와줄 알고리즘

In [24]:
model = DecisionTreeClassifier()
model.fit(X_train, Y_train) # 학습 수행 -> Model 변수에 학습 수식 저장

DecisionTreeClassifier()

In [25]:
# 5. 평가 수행
from sklearn.metrics import accuracy_score

In [26]:
# 생성한 모델에 X 데이터를 넣어 모델이 계산한 예측값 확인
Y_train_pred = model.predict(X_train) # 학습 X 데이터를 이용해 예측

In [27]:
# 학습 능력 평가
accuracy_score(Y_train, Y_train_pred)

0.9883877316713138

In [28]:
# 생성한 모델에, 검증 X 데이터를 넣어 예측값 계산
Y_test_pred = model.predict(X_test)

In [29]:
# 일반화 능력 평가
accuracy_score(Y_test, Y_test_pred)

0.9860263727612675

In [30]:
# 분류모델 평가지표 계산함수
from sklearn.metrics import classification_report

In [31]:
# 학습능력평가
print(classification_report(Y_train, Y_train_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     30091
           1       0.95      0.11      0.19       394

    accuracy                           0.99     30485
   macro avg       0.97      0.55      0.59     30485
weighted avg       0.99      0.99      0.98     30485



In [32]:
# 일반화능력평가
print(classification_report(Y_test, Y_test_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     10046
           1       0.07      0.02      0.03       116

    accuracy                           0.99     10162
   macro avg       0.53      0.51      0.51     10162
weighted avg       0.98      0.99      0.98     10162



In [35]:
# 6. 실행 / 학습한 모델을 파일형태로 저장
import pickle

In [36]:
# 학습모델을 저장
pickle.dump(model, open('model.sav', 'wb'))

**저장한 모델 불러와서 적용하기**

In [ ]:
# 모델을 불러오기
model = pickle.load(open('model.sav', 'rb'))
model

In [ ]:
# 새로운 값 입력하기 (input())
x1 = input('계약 기간을 입력하시오 : ')
x2 = input('월렌탈비용을 입력하시오 : ')
x3 = input('고객 연령을 입력하시오 : ')
x4 = input('고객 신용 등급을 입력하시오 : ')

input_data = [x1, x2, x3, x4]

In [ ]:
# 입력받은 개별 값을 데이터 프레임 형태로 변환
new_input = pd.DataFrame(data = [input_data],
                         columns = ['Term', 'Amount_Month', 'Age', 'Credit_Rank'])

In [ ]:
# 입력된 값을 이용해, 예측 수행
model.predict(new_input)